# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
import re

In [2]:
dbName = "datamad0320" # Name of the database I've created
mongodbURL = f"mongodb://localhost/{dbName}" #url of the database (placed in my local)
client = MongoClient(mongodbURL)

In [3]:
# Checking connection
#client.server_info()

In [4]:
db = client.get_database() # Gets all the databases from the client
cursor = db.companies # Selecting the companies database

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [5]:
myquery = {"name": "Babelgum"}

# En MongoDB Compass:
# FILTER: {name : "Babelgum" } # Encuentra las empresas que se llamen Babelgum
# FILTER: {name : /Babelgum/i } # Encuentra todas las empresas que contengan Babelgum (case insensitive)

In [6]:
cursor = db.companies.find(myquery)
for e in cursor:
    print(e["name"])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [7]:
myquery = {"number_of_employees": {"$gt": 5000} }
fields = {"_id":0,"name":1, "number_of_employees":1}

In [8]:
#list(db.companies.find(myquery, fields).limit(20).sort("number_of_employees", ASCENDING)) # Option A
list(db.companies.find(myquery, fields).limit(20).sort("number_of_employees", 1)) # Option B

[{'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Hexaware Technologies', 'number_of_employees': 5200},
 {'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'OpenText', 'number_of_employees': 5300},
 {'name': 'LSI', 'number_of_employees': 5400},
 {'name': 'CPM Braxis', 'number_of_employees': 5400},
 {'name': 'Microchip Technologies', 'number_of_employees': 5500},
 {'name': 'Mediaset', 'number_of_employees': 5729},
 {'name': 'Mindray Medical International', 'number_of_employees': 5763},
 {'name': 'Baidu', 'number_of_employees': 6000},
 {'name': 'Dentsu', 'number_of_employees': 6000},
 {'name': 'Tata Communications', 'number_of_employees': 6000},
 {'name': 'Atmel', 'number_of_employees': 6000},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'SRA International', 'number_of_employees': 6400},
 {'name': "Moody's", 'number_of_employees': 6800},
 {'name': 'ManTech', 'number_of_employees': 7000},
 {'name': 'Ad

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [9]:
myquery = { "founded_year": { "$in": [ 2000, 2001, 2002, 2003, 2004, 2005 ] } } # Option A
#myquery = { "$and": [{"founded_year": {"$gt":1999}}, {"founded_year": {"$lt":2006}}]} # Option B
#myquery = { "founded_year": { "$in": [{"$range": [2000, 2005]}] } } -->> No funciona

fields = {"_id":0,"name":1, "founded_year":1}

In [10]:
# Showing only the first 10 companies 
list(db.companies.find(myquery, fields).limit(10))

[{'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [11]:
myquery = { "$and": [ {"ipo.valuation_amount": {"$gt": 100000000}}, {"founded_year": {"$lt":2010}}]}
fields = {"_id":0,"name":1, "ipo":1}

In [12]:
# Showing only the first 3 companies 
list(db.companies.find(myquery, fields).limit(3))

[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [13]:
myquery = {"$and": [{"number_of_employees": {"$lt": 1000}}, {"founded_year": {"$lt": 2005}} ]}
fields = {"_id":0,"name":1, "founded_year":1, "number_of_employees":1}

In [14]:
list(db.companies.find(myquery, fields).limit(10).sort("number_of_employees", DESCENDING)) # Option A
#list(db.companies.find(myquery, fields).limit(10).sort("number_of_employees", -1)) # Option B

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Yelp', 'number_of_employees': 800, 'founded_year': 2004},
 {'name': 'ZoomInfo', 'number_of_employees': 800, 'founded_year': 2000},
 {'name': 'MySpace', 'number_of_employees': 800, 'founded_year': 2003}]

### 6. All the companies that don't include the `partners` field.

In [15]:
myquery = {"partners": { "$exists": True }}
fields = {"_id":0,"name":1}

In [16]:
# Option A: creating a list from the cursor showing the name of the companies and calculating the length.
#len(list(db.companies.find(myquery, fields))) 

# Option B: Count number of companies that meet this condition
db.companies.count_documents(myquery)

18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [17]:
myquery = {"category_code": { "$type": "null" }}

fields = {"_id":0,"name":1}

In [18]:
# Option A: creating a list from the cursor showing the name of the companies and calculating the length.
#len(list(db.companies.find(myquery, fields))) 

# Option B: Count number of companies that meet this condition
db.companies.count_documents(myquery)

2751

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [19]:
myquery = {"$and": [{"number_of_employees": {"$gt": 100}}, {"number_of_employees": {"$lt": 1000}} ]}
fields = {"_id":0,"name":1, "number_of_employees":1}

In [20]:
list(db.companies.find(myquery, fields))[0:10]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [21]:
fields = {"_id":0, "name":1, "ipo.valuation_amount":1}

In [22]:
# Showing only the 10 first companies
list(db.companies.find({},fields).limit(10).sort("ipo.valuation_amount", DESCENDING))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [23]:
fields = {"_id":0, "name":1, "number_of_employees":1}

In [24]:
# Showing only the 10 first companies
list(db.companies.find({}, fields).limit(10).sort("number_of_employees", DESCENDING))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [25]:
myquery = {"founded_month": {"$gt": 5}}
fields = {"_id":0, "name":1, "founded_month":1}

In [26]:
# Searching 1000 companies but showing only 10
list(db.companies.find(myquery, fields).limit(1000))[0:10]

[{'name': 'Zoho', 'founded_month': 9},
 {'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [27]:
myquery = {"$and": [{"founded_year": {"$lt": 2000}}, {"acquisition.price_amount": {"$gt": 10000000}}]}
fields = {"_id":0,"name":1, "founded_year":1, "acquisition.price_amount":1}

In [28]:
list(db.companies.find(myquery, fields))[0:5]

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [29]:
myquery = {"acquisition.acquired_year": {"$gt": 2010}}
fields = {"_id":0,"name":1, "acquisition":1}

In [30]:
# Showing only the first one
list(db.companies.find(myquery, fields).sort("acquisition.amount", DESCENDING))[0:1]

[{'name': 'Wetpaint',
  'acquisition': {'price_amount': 30000000,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': 'http://allthingsd.com/20131216/viggle-tries-to-bulk-up-its-social-tv-business-by-buying-wetpaint/?mod=atdtweet',
   'source_description': ' Viggle Tries to Bulk Up Its Social TV Business by Buying Wetpaint',
   'acquired_year': 2013,
   'acquired_month': 12,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Viggle', 'permalink': 'viggle'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [39]:
#myquery = {"founded_year": {"$exists": True, "$ne": None}} # Option A
myquery = {"founded_year": {"$ne": None}} # Option B
# In MongoDB Compass: {"founded_year": {$ne: null}}

fields = {"_id":0,"name":1, "founded_year":1}

In [40]:
list(db.companies.find(myquery, fields).limit(10).sort("founded_year", ASCENDING))

[{'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'McKesson', 'founded_year': 1833},
 {'name': 'Bachmann Industries', 'founded_year': 1833},
 {'name': 'Bertelsmann', 'founded_year': 1835},
 {'name': 'Accuity', 'founded_year': 1836},
 {'name': 'CENTRA', 'founded_year': 1839},
 {'name': 'WeGame', 'founded_year': 1840}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [41]:
myquery = {"founded_day": {"$lt": 8}}
fields = {"_id":0,"name":1, "founded_day":1}

In [42]:
list(db.companies.find(myquery, fields).limit(10).sort("acquisitions.price_amount", DESCENDING))

[{'name': 'Google', 'founded_day': 7},
 {'name': 'Microsoft', 'founded_day': 4},
 {'name': 'Yahoo!', 'founded_day': 1},
 {'name': 'Juniper Networks', 'founded_day': 6},
 {'name': 'FlowPlay', 'founded_day': 1},
 {'name': 'eBay', 'founded_day': 1},
 {'name': 'IAC', 'founded_day': 1},
 {'name': 'OpenText', 'founded_day': 6},
 {'name': 'Facebook', 'founded_day': 1},
 {'name': 'PayPal', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [43]:
myquery = {"$and": [{"category_code": "web"}, {"number_of_employees": {"$gt":4000}} ]}
fields = {"_id":0,"name":1, "number_of_employees":1}

In [44]:
list(db.companies.find(myquery, fields).sort("number_of_employees", ASCENDING))[0:10]

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [45]:
myquery = {"$and": [{
                    "acquisition.price_amount": {"$gt": 10000000}}, 
                    {"acquisition.price_currency_code": "EUR"}]}
fields = {"_id":0,"name":1, "acquisition.price_amount":1, "acquisition.price_currency_code":1}

In [46]:
list(db.companies.find(myquery, fields))[0:10]

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [49]:
myquery = {"acquisition.acquired_month": {"$lt": 4}}
fields = {"_id":0,"name":1, "acquisition":1}

In [51]:
list(db.companies.find(myquery, fields).limit(10))[0]

{'name': 'Kyte',
 'acquisition': {'price_amount': None,
  'price_currency_code': 'USD',
  'term_code': None,
  'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
  'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
  'acquired_year': 2011,
  'acquired_month': 1,
  'acquired_day': 31,
  'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}}

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [54]:
myquery = {"$and": [{"founded_year": {"$gt": 1999}}, 
                    {"founded_year": {"$lt": 2011}}, 
                    {"acquisition.acquired_year": {"$gt": 2011}}]}
fields = {"_id":0,"name":1, "founded_year":1, "acquisition.acquired_year":1}

In [55]:
list(db.companies.find(myquery, fields).limit(10).sort("founded_year", ASCENDING))

[{'name': 'SelectMinds',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'SnapNames',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Handmark',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Bestofmedia Group',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Axis',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Axis',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'ExactTarget',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Rainbow Partners',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Crossbeam Systems',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'BroadLight',
  'founded_year': 2000,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [58]:
myquery = {"$subtract": [ "deadpooled_year", "founded_year" ]}

In [71]:
list(db.companies.find( { "$where": '("deadpooled_year" - "founded_year") > 3'}))

[]